In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Importación de Librerias

In [ ]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report,confusion_matrix
%matplotlib inline

In [ ]:
df1 = pd.read_csv('/kaggle/input/credit-card-customers/BankChurners.csv')
df1.head()

Limpiamos las etiquetas o features que no nos interesan.

In [ ]:
df = df1.iloc[:, :-2]
df.drop(['CLIENTNUM','Total_Ct_Chng_Q4_Q1','Total_Amt_Chng_Q4_Q1'], axis=1, inplace=True)
df.head()

# EDA

In [ ]:
df.dtypes

In [ ]:
df.describe()

In [ ]:
print(df.isnull().sum())

In [ ]:
df['Attrition_Flag'].value_counts()

In [ ]:
plt.figure(figsize=(20,10))
c= df.corr()
sns.heatmap(c,cmap='BrBG',annot=True)
c

In [ ]:
plt.figure(figsize=(10,6))
df[df['Attrition_Flag']=='Existing Customer']['Total_Trans_Ct'].hist(alpha=0.5,color='blue',
                                              bins=30,label='Cliente Existente')
df[df['Attrition_Flag']=='Attrited Customer']['Total_Trans_Ct'].hist(alpha=0.5,color='red',
                                              bins=30, label='Cliente Perdido')
plt.legend()
plt.xlabel('Tipo Cliente')
plt.ylabel('Total de Transacciones (12 meses)')

In [ ]:
plt.figure(figsize=(11,7))
sns.lmplot(y='Total_Trans_Ct',x='Total_Trans_Amt',data=df,hue='Gender',
           col='Attrition_Flag',palette='Set1')

# Decision Tree

Vamos a hacer el arbol de desición con la categoría de cliente, según si lo sigue siendo o ya no. Pero antes debemos transformar las variables categóricas, para ellos primero las identificaremos y luego aplicamos la función LabelEncoder

In [ ]:
print(df.columns[df.dtypes==object].tolist())

In [ ]:
from sklearn.preprocessing import LabelEncoder 

class MultiColumnLabelEncoder: 
    def __init__(self,columns = None): 
     self.columns = ['Attrition_Flag', 'Gender', 'Education_Level', 'Marital_Status', 'Income_Category', 'Card_Category']

    def fit(self,X,y=None): 
     return self 

    def transform(self,X): 
     ''' 
Transforma las columnas especificas en el df en columnas en código.
     ''' 
     output = df.copy() 
     if self.columns is not None: 
      for col in self.columns: 
       output[col] = LabelEncoder().fit_transform(output[col]) 
     else: 
      for colname,col in output.iteritems(): 
       output[colname] = LabelEncoder().fit_transform(col) 
     return output 

    def fit_transform(self,X,y=None): 
     return self.fit(X,y).transform(X) 

In [ ]:
df_final=MultiColumnLabelEncoder(columns = ['Attrition_Flag', 'Gender',  
                                               'Education_Level', 'Marital_Status', 
                                               'Income_Category', 'Card_Category']).fit_transform(df)
df_final
#Ahora todas las variables son numéricas

In [ ]:
X = df_final.drop('Attrition_Flag',axis=1)
y = df_final['Attrition_Flag']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30)

In [ ]:
dtree = DecisionTreeClassifier()
dtree.fit(X_train,y_train)

# Predicciones y Evaluación del Modelo

In [ ]:
predictions = dtree.predict(X_test)
print(classification_report(y_test,predictions))


Con una exactitud (accuracy) en torno al 93%, una precisión y un recall cercano al 88% podemos decir que las predicciones son bastante buenas.

In [ ]:
print(confusion_matrix(y_test,predictions))

Finalmente, hacemos una lista de la importancia o participación de cada etiqueta:

In [ ]:
etiq = dict(zip(X.columns, dtree.feature_importances_))

orden = sorted(etiq.items(), key=lambda X: X[1], reverse=True)

pd.DataFrame(orden, columns=['Etiqueta', 'Importancia'])

# Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
rfc = RandomForestClassifier(n_estimators=600)

In [ ]:
rfc.fit(X_train,y_train)

In [ ]:
predictions = rfc.predict(X_test)

# Predicciones y Evaluación del Modelo RFC


In [ ]:
print(classification_report(y_test,predictions))

In [ ]:
print(confusion_matrix(y_test,predictions))

Vemos que el Randfom Forest tiene una precisión y un ajuste mejor que el Arbol de Desición.

In [ ]:
etiq = dict(zip(X.columns, rfc.feature_importances_))

orden = sorted(etiq.items(), key=lambda X: X[1], reverse=True)

pd.DataFrame(orden, columns=['Etiqueta', 'Importancia'])

Finalmente, los resultados de importancia de las etiquetas cambia respecto al arbol de desición, tanto en preponderancia como en las labels elegidas. Ahora según el rfc debemos elegir el monto total de transacciones como nodo principal en lugar de la cantidad total de transacciones.